In [11]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
from math import floor
import pickle
from pymongo import MongoClient
from os import listdir
from os.path import isfile, join, isdir
from os import listdir
import re
import pickle
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
import pydblite
import warnings
class dbConf(object):
    # connect to database
    client = MongoClient('mongodb://localhost:27017/')
    try:
        mdb = client.racmo
        Notifications = mdb.Notifications
        PdfFiles=mdb.PdfFiles
        Keywords=mdb.Keywords
    except Exception as e:
        print(e)

# PDF_DIR = "/home/thrymr/Racmo/processed/Final version of Notifications/"

In [12]:
KeywordDir="/home/thrymr/Racmo/Notification Keywords/"
keyword_files = [join(KeywordDir,f) for f in listdir(KeywordDir)\
                 if isfile(join(KeywordDir, f)) and 
                 f[-4:].lower() == "xlsx" and f[:2]!='N8']
ls1, ls2 ,ls3 = [], [],[]
s1=[]
for f in keyword_files:
    xl_file = pd.ExcelFile(f)
    dfs = xl_file.parse(xl_file.sheet_names[0])
    
    tic=False
    
    for index, row in dfs.iterrows():
        ite=dict()
        if(row[0]=='TICKETS' or row[0]=='TICKET'):
            tic=True
        if(row[0]!='TICKETS' and row[0]!='TICKET' and isinstance(row[0], basestring)):
        
            ite['file_class']=f.split('/')[-1].split()[0]
            s1.append(ite['file_class'])
            ite['notification_type']=row[0]
            if(tic):
                ite['file_type']='TICKET'
            else:
                ite['file_type']='NOTIFICATION'
            kw=list()
            for i in range(1,row.size):
                if(isinstance(row[i], basestring) and row[i]!='+'):
                    kw.append(unidecode.unidecode(row[i]).lower() )
            ite['keyword']=kw
            if(ite['file_class'] == 'NX'):
                ls3.append(ite)
            elif (ite['file_class'] == 'N-1'):
                ls2.append(ite)
            else:
                ls1.append(ite)
print(set(s1))
newkdf=pd.DataFrame(ls1)
remkdf=pd.DataFrame(ls2)
suspkdf=pd.DataFrame(ls3)

# newkdf

set(['N13', 'N10', 'N11', 'N16', 'N14', 'N-1', 'NX', 'N9', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7'])


In [13]:
suspkdf=suspkdf.rename(columns={"notification_type":"remove_class"})

In [14]:
notification_corelation_dict = { 'N1' : {'N1','N4','N7','N11','N13'},
                   'N2' : {'N2','N4','N7','N8','N11','N13','N15','N16'},
                   'N3' : {'N3','N4','N7','N8','N11','N13','N15','N16'},
                   'N4' : {'N1','N2','N3','N4','N7','N8','N9','N10','N13','N14','N15','N16'},
                   'N5' : {'N5'},
                   'N6' : {'N6'},
                   'N7' : {'N1','N2','N3','N4','N7','N11'},
                   'N8' : {'N2','N3','N4','N8','N9','N10','N11'},
                   'N9' : {'N4','N8','N9','N10','N11','N13','N15','N16'},
                   'N10' : {'N4','N8','N9','N10','N11','N12','N13','N15','N16'},
                   'N11' : {'N1','N2','N3','N7','N8','N9','N10','N11','N13','N14','N15','N15'},
                   'N12' : {'N12'},
                   'N13' : {'N1','N2','N3','N4','N9','N10','N11','N13'},
                   'N14' : {'N4','N11','N14'},
                   'N15' : {'N2','N3','N4','N9','N10','N11','N15'},
                   'N16' : {'N2','N3','N4','N9','N10','N11','N16'}
        } 

In [15]:
pdfclass={ 'N1':'CLAIM ADMITTED TO PROCEDURE','N2':'ACCEPTED TRANSFER OF LEGAL REPRESENTATION',
           'N3':'ENFORCEMENT ORDER','N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS','N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING','N8':'ASSET INQUIRY','N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT','N11':'BANK TRANSFERS','N12':'SOLICITOR REMOVAL',
           'N13':'SUSPENDED HEARINGS','N14':'CANCELLED HEARINGS','N15':'NEGATIVE ASSET INQUIRY','N16':'PENDING ASSET INQUIRY'}

newkdf['purpose']=''
n=newkdf.iterrows()
for i,row in n:
    if 'EJECUCION DE TITULOS NO JUDICIALES'.lower() in row['keyword']:
        newkdf=newkdf.drop(newkdf.index[i])
    elif(not ( row['file_class'] =='N5'or row['file_class'] =='N6')):
        if(pdfclass[row['file_class']] in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N5':
        if('N5 - REJECTED CLAIMS' in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N6':
        if('N6 - REJECTED TRANSFER OF LEGAL REPRESENTATION' in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
            
k=newkdf[(newkdf['file_class']=='N10') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')].groupby('notification_type')
print(k.groups)
new10df=pd.DataFrame(columns=newkdf.columns)
i=0
for lidx in k.groups['N10 - REQUIREMENTS']:
    for ridx in k.groups['N10 - REQUIREMENT SPECIFICATION']:
        new10df=new10df.append(newkdf.iloc[lidx],ignore_index=True)
        
        new10df.loc[i,'keyword']=new10df.loc[i,'keyword']+newkdf.loc[ridx,'keyword']
        i=i+1
newkdf = newkdf[~((newkdf['file_class']=='N10') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')&
                  ((newkdf['notification_type']=='N10 - REQUIREMENTS')|(newkdf['notification_type']=='N10 - REQUIREMENT SPECIFICATION')))].append(new10df,ignore_index=True)
newkdf = newkdf.rename(columns={'file_class':'fileclass','file_type':'filetype',"notification_type":"decision_type"})
remkdf = remkdf.rename(columns={'file_class':'remove','file_type':'filetype',"notification_type":"fileclass"})


{u'N10 - REQUIREMENT SPECIFICATION': Int64Index([ 15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
             28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
             41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
             54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
             67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
             80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
             93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
            106, 107, 108],
           dtype='int64'), u'N10 - FULL REQUIREMENT SPECIFICATION': Int64Index([109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
            122, 123, 124, 125, 126, 127],
           dtype='int64'), u'N10 - REQUIREMENTS': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')}


In [16]:
warnings.simplefilter('ignore')
kdf=newkdf
kdf['keywordHash'] = kdf['keyword'].apply(lambda x :hashlib.sha256(json.dumps(x)).hexdigest())
kdf['sub'] = kdf['keyword'].apply(lambda x :[])
kdf['bias']=kdf['fileclass']
kdf.loc[(kdf['fileclass']=='N2')&(kdf['filetype']=='NOTIFICATION'),'bias']='N12'
def add_sub_link_keyword(kdf,keywords,fileclass,filetype,purpose,decision_type,bias,parent_class):
    n_ind = kdf.index[-1]+1
    for n15k in keywords:
        k = [unidecode.unidecode(unicode(n15k,'UTF-8')).lower()]
        hsh=hashlib.sha256(json.dumps(k)).hexdigest()
        
        if not hsh in kdf['keywordHash'].values.tolist():
            kdf.set_value(n_ind,'fileclass',fileclass)
            kdf.set_value(n_ind,'filetype',filetype)
            kdf.set_value(n_ind,'purpose',purpose)
            kdf.set_value(n_ind,'decision_type',decision_type)

            kdf.set_value(n_ind,'keyword',k)
            kdf.set_value(n_ind,'bias',bias)
            kdf.set_value(n_ind,'keywordHash',hsh)
            kdf.set_value(n_ind,'sub',[])
        for i in kdf[(kdf['fileclass']==parent_class)&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:
            
            kx = kdf[kdf.index==i]['sub'].values[0]
            kx.append(hsh)
            kdf.set_value(i,'sub',kx)
        n_ind+=1
    return kdf
N14_from_N13 = ['SE DECLARA TERMINADO','TENER POR TERMINADO','DESISTIMIENTO DEL PROCESO',
 'SE DECRETA LA TERMINACION DEL PROCESO','HABIENDOSE SOLICITADO DESISTIMIENTO','HABER LLEGADO A UN ACUERDO']
#Parent->N2,Notification,classification
N2_solicitor_list=["MALAGON LOYO","ALCOCER ANTON","GARCIA ABASCAL","GARCÍA ABASCAL"]
#Parent->N4,Notification,classification
N6_from_N2=["NO HA LUGAR A TENER POR PERSONADA","NO HA LUGAR A LO SOLICITADO"]
N2_from_N6=["EN ESTE CASO HABIÉNDOSE JUSTIFICADO DE MODO FEHACIENTE LA REALIDAD DE LA CESIÓN"]
N9_from_N10=["APORTE NOTA SIMPLE DEL REGISTRO MERCANTIL","ES DESCONOCIDO EN ESTE DOMICILIO"]
N11_from_N4 = ['TRANSFERENCIA','EL No DE CUENTA DESIGNADO','A LA CUENTA BANCARIA DESIGNADA',
                        'MEDIANTE TRANSFERENCIA A LA CUENTA DESIGNADA','MEDIANTE TRANSFERENCIA','SERA TRANSFERIDO']
#Parent->N16,Notification,classification
N8_from_N16 = ['RESUMEN DEL RESULTADO']
N10_from_N11=['REQUIÉRASE A LA PARTE EJECUTANTE A FIN DE QUE DESIGNE CUENTA BANCARIA']
#Parent->N16,Notification,classification
N15_from_N16 = ['CONSULTA INTEGRAL','RESUMEN DE INCIDENCIAS O INFORMACION NO ENCONTRADA']
N5_to_N10=['BAJO APERCIBIMIENTO DE QUE DE NO VERIFICARSE','SE ACUERDA LA PRÓRROGA DEL PLAZO','SE PROCEDERÁ A LA INADMISIÓN A TRÁMITE']
N4_from_N11=['CUENTA DE FONDOS PROVISIONALMENTE ABANDONADOS','MANDAMIENTO DE PAGO','MANDAMIENTO DE DEVOLUCIÓN','MANDAMIENTO TELEMÁTICO DE PAGO']
kdf = add_sub_link_keyword(kdf,N14_from_N13,'N14','NOTIFICATION','CLASSIFICATION_2','N14-CANCELLED HEARINGS','N14','N13')
kdf = add_sub_link_keyword(kdf,N6_from_N2,'N6','NOTIFICATION','CLASSIFICATION_2','N6-REJECTED TRANSFER OF LEGAL REPRESENTATION','N6','N2')
kdf = add_sub_link_keyword(kdf,N2_from_N6,'N2','NOTIFICATION','CLASSIFICATION_2','N2- TRANSFER OF LEGAL REPRESENTATION','N2','N6')
kdf = add_sub_link_keyword(kdf,N9_from_N10,'N9','NOTIFICATION','CLASSIFICATION_2','N9- PLACE OF RESIDENCY REQUIRED','N9','N10')

kdf = add_sub_link_keyword(kdf,N2_solicitor_list,'N2','NOTIFICATION','AUXILIARY','N2-SOLICITOR LIST','N2','N2')
kdf = add_sub_link_keyword(kdf,N11_from_N4,'N11','NOTIFICATION','CLASSIFICATION_2','N11-BANK TRANSFERS','N11','N4')
kdf = add_sub_link_keyword(kdf,N5_to_N10,'N10','NOTIFICATION','CLASSIFICATION_2','N10-REQUIREMENTS','N10','N5')
kdf = add_sub_link_keyword(kdf,N4_from_N11,'N4','NOTIFICATION','CLASSIFICATION_2','N4-STATEMENT OF PAYMENT','N4','N11')
kdf = add_sub_link_keyword(kdf,N10_from_N11,'N10','NOTIFICATION','CLASSIFICATION_2','N10-REQUIREMENTS','N10','N11')



In [17]:

for k1 in N6_from_N2:
    k = [unidecode.unidecode(unicode(k1,'UTF-8')).lower()]
    hsh=hashlib.sha256(json.dumps(k)).hexdigest()
    for i in kdf[(kdf['fileclass']=='N2')&(kdf['purpose']=='AUXILIARY')].index:
            kx = kdf[kdf.index==i]['sub'].values[0]
            kx.append(hsh)
            kdf.set_value(i,'sub ',kx)

keys = [[unidecode.unidecode(unicode('RESUMEN DEL RESULTADO','UTF-8')).lower()],[unidecode.unidecode(unicode('CONSULTA DE SITUACIONES LABORALES DE LA TGSS','UTF-8')).lower(),unidecode.unidecode(unicode('SITUACION ACTUAL: ALTA','UTF-8')).lower()]]
for k in keys:
    hsh8=hashlib.sha256(json.dumps(k)).hexdigest()
    n_ind = kdf.index[-1]+1
    if not hsh8 in kdf['keywordHash'].values.tolist():
        kdf.set_value(n_ind,'fileclass','N8')
        kdf.set_value(n_ind,'filetype','NOTIFICATION')
        kdf.set_value(n_ind,'purpose','CLASSIFICATION')
        kdf.set_value(n_ind,'decision_type','N8-ASSET INQUIRY')

        kdf.set_value(n_ind,'keyword',k)
        kdf.set_value(n_ind,'bias','N8')
        kdf.set_value(n_ind,'keywordHash',hsh8)
        kdf.set_value(n_ind,'sub',[])
    for i in kdf[(kdf['fileclass']=='N16')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hsh8)
        kdf.set_value(i,'sub',kx)
keys = [[unidecode.unidecode(unicode('CONSULTA INTEGRAL','UTF-8')).lower(),unidecode.unidecode(unicode('RESUMEN DE INCIDENCIAS O INFORMACION NO ENCONTRADA','UTF-8')).lower()],[unidecode.unidecode(unicode('CONSULTA DE SITUACIONES LABORALES DE LA TGSS','UTF-8')).lower(),unidecode.unidecode(unicode('SITUACION ACTUAL: BAJA','UTF-8')).lower()]]
for k in keys:
    hsh=hashlib.sha256(json.dumps(k)).hexdigest()
    n_ind = kdf.index[-1]+1
    if not hsh in kdf['keywordHash'].values.tolist():
        kdf.set_value(n_ind,'fileclass','N15')
        kdf.set_value(n_ind,'filetype','NOTIFICATION')
        kdf.set_value(n_ind,'purpose','CLASSIFICATION_2')
        kdf.set_value(n_ind,'decision_type','N15-NEGATIVE ASSET INQUIRY')

        kdf.set_value(n_ind,'keyword',k)
        kdf.set_value(n_ind,'bias','N15')
        kdf.set_value(n_ind,'keywordHash',hsh)
        kdf.set_value(n_ind,'sub',[hsh8])
    for i in kdf[(kdf['fileclass']=='N16')&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hsh8)
        kdf.set_value(i,'sub',kx)
for hsh in kdf[(kdf['fileclass']=='N13')&(kdf['filetype']=='NOTIFICATION')]['keywordHash'].values.tolist():
    for i in kdf[(kdf['fileclass']=='N7')&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hsh)
        kdf.set_value(i,'sub',kx) 
for hs in kdf[(kdf['fileclass']=='N5')&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')]['keywordHash'].values.tolist():
    for i in kdf[(kdf['fileclass']=='N1')&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hs)
        kdf.set_value(i,'sub',kx) 

In [18]:
kdf['keyword'] = kdf['keyword'].apply(lambda x : json.dumps(x))
kg=kdf[kdf['purpose']=='CLASSIFICATION'].groupby('keyword')
ls=[]
for k,v in kg.groups.items():
    if(len(v)>1):
        if len(set(kdf[kdf['keyword']==k]['fileclass'].values)) ==1:
            ls=ls+v.values.tolist()[1:]
        else:
            if set(kdf[kdf['keyword']==k]['fileclass'].values)<=set(['N8','N15','N16']):
                for j in v.values.tolist():
                    if kdf.loc[j,'fileclass']!='N16':
                        ls.append(j)
            elif set(kdf[kdf['keyword']==k]['fileclass'].values)<=set(['N13','N14']):
                for j in v.values.tolist():
                    if kdf.loc[j,'fileclass']!='N13':
                        ls.append(j)
print(ls)
kdf['keyword'] = kdf['keyword'].apply(lambda x : json.loads(x))

kdf=kdf.drop(kdf.index[ls])

[83, 16, 545, 81, 82, 572, 543, 542, 583, 567, 563, 397, 445, 544, 541]


In [19]:

a = {'keywords':kdf,
        'susKeyword':suspkdf
    }
with open('Keyword_Analysis2.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
# a = {'keywordsX':kxddf}
# with open('Keyword_Analysis.pickle', 'wb') as handle:
#     pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
kdf

,fileclass,filetype,keyword,decision_type,purpose,keywordHash,sub,bias,sub
0,N10,NOTIFICATION,[atender requirimiento],N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,efb41a5261c3eb506f581711edf5be25fd7c3d10f3b362...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
1,N10,NOTIFICATION,[se requiere nuevamente],N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,9804e4de3cb435d4b4d045a020a9470e8baaa05350c3fb...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
2,N10,NOTIFICATION,"[se concede, el plazo, con el fin de que]",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,ed7b6a143589ebe7c8233a4e29e2e10909014e9a6479c2...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
3,N10,NOTIFICATION,"[se concede un nuevo plazo, documentacion requ...",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,c7414ecedce1a02cebe54ab95c08f526955656fe342d4b...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
4,N10,NOTIFICATION,"[manifestar, la ultima cuota]",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,49503fdab4c38c81736bcfef7397332823ffbfecb1a7d4...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
5,N10,NOTIFICATION,"[se acuerda prorroga, para aportar contrato]",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,f7f71d65a1d17d054559b35385e3e005844f64c56cd014...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
6,N10,NOTIFICATION,"[se observa que el nie, pertenece a, y no a]",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,af9de43afe6013a551e4a2f37fdfbe591e6d43bcbf5cc5...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
7,N10,NOTIFICATION,"[oigase, las partes]",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,bd2639d7bb4472bc252740f3f9b08fa879877974fea682...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
8,N10,NOTIFICATION,[hasta que sean subsanados los defectos apreci...,N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,fc36c66a438dc130995dc9ea56cee7b1881bda9cc5330e...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN
9,N10,NOTIFICATION,"[a la parte ejecutante, a fin de que alegue lo...",N10 - FULL REQUIREMENT SPECIFICATION,CLASSIFICATION,ed2841d2dcf304ff82119e22dc6b7b6dc617b17d9f22f3...,[e53e5e18c89745e241b161e9453efebbd2808050437fb...,N10,NaN


In [20]:
suspkdf

,file_class,file_type,keyword,remove_class
0,NX,NOTIFICATION,[archivar el presente procedimiento],N1
1,NX,NOTIFICATION,[se acuerda el archivo definitivo],N1
2,NX,NOTIFICATION,[se acuerda el sobreseimiento],N1
3,NX,NOTIFICATION,[se declara finalizado el presente proceso mon...,N1
4,NX,NOTIFICATION,[continuandose la ejecucion],N1
5,NX,NOTIFICATION,[dar por terminado el presente expediente],N1
6,NX,NOTIFICATION,[dando publicidad a la conclusion y archivo],N1
7,NX,NOTIFICATION,[archivase el expediente],N1
8,NX,NOTIFICATION,[y dimana del juicio monitorio],N1
9,NX,NOTIFICATION,[admitida a tramite la ejecucion],N1
